In [ ]:
import os, random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from IPython.display import display

# sklearn (tabular)
from sklearn.model_selection import KFold
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import Ridge, LinearRegression

# torch (cnn)
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 220)

INPUT_DIR = Path("/kaggle/input/csiro-biomass")
subdirs = [p for p in INPUT_DIR.iterdir() if p.is_dir()]
COMP_DIR = subdirs[0] if len(subdirs) == 1 else INPUT_DIR
print("Usando carpeta de datos:", COMP_DIR)

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
train = pd.read_csv(COMP_DIR / "train.csv")
test  = pd.read_csv(COMP_DIR / "test.csv")

print("train shape:", train.shape)
print("test shape:", test.shape)
display(train.head())
display(test.head())

In [ ]:
target_names = sorted(train["target_name"].unique())
print("Targets:", target_names, "| n_targets:", len(target_names))

y_wide = (train.pivot(index="image_path", columns="target_name", values="target")
               .loc[:, target_names])
print("y_wide shape:", y_wide.shape)

TARGET_WEIGHTS = {
    "Dry_Green_g": 0.1,
    "Dry_Dead_g":  0.1,
    "Dry_Clover_g":0.1,
    "GDM_g":       0.2,
    "Dry_Total_g": 0.5,
}
W_VEC = np.array([TARGET_WEIGHTS[t] for t in target_names], dtype=np.float32)

def weighted_r2_from_long(df_long_true_pred,
                          target_col="target_name", y_col="y_true", yhat_col="y_pred"):
    w = df_long_true_pred[target_col].map(TARGET_WEIGHTS).values.astype(np.float64)
    y  = df_long_true_pred[y_col].values.astype(np.float64)
    yh = df_long_true_pred[yhat_col].values.astype(np.float64)
    yw = np.sum(w * y) / np.sum(w)
    ss_res = np.sum(w * (y - yh)**2)
    ss_tot = np.sum(w * (y - yw)**2)
    return 1.0 - ss_res / (ss_tot + 1e-12)

In [ ]:
def extract_image_features(rel_path: str) -> dict:
    img_path = COMP_DIR / rel_path
    feats = {"mean_R":0.0,"mean_G":0.0,"mean_B":0.0,
             "std_R":0.0,"std_G":0.0,"std_B":0.0,
             "excess_green":0.0,"mean_gray":0.0,
             "mean_g_fraction":0.0,"prop_green_pixels":0.0,"p90_excess_green":0.0,
             "mean_H":0.0,"mean_S":0.0,"mean_V":0.0,
             "std_H":0.0,"std_S":0.0,"std_V":0.0,
             "mean_L":0.0,"mean_A":0.0,"mean_Blab":0.0,
             "std_L":0.0,"std_A":0.0,"std_Blab":0.0,
             "edge_density":0.0,"lap_var":0.0,"entropy":0.0}
    try:
        with Image.open(img_path) as img:
            img = img.convert("RGB")
            arr = np.asarray(img).astype(np.float32)
        R,G,B = arr[:,:,0], arr[:,:,1], arr[:,:,2]
        feats["mean_R"],feats["mean_G"],feats["mean_B"] = R.mean(),G.mean(),B.mean()
        feats["std_R"],feats["std_G"],feats["std_B"]    = R.std(),G.std(),B.std()
        eg = 2*G - R - B; feats["excess_green"] = eg.mean()
        gray = 0.299*R + 0.587*G + 0.114*B; feats["mean_gray"] = gray.mean()
        denom = R+G+B+1e-6; feats["mean_g_fraction"] = (G/denom).mean()
        feats["prop_green_pixels"] = ((G>R)&(G>B)).mean()
        feats["p90_excess_green"] = float(np.percentile(eg,90))
        hsv = np.array(Image.fromarray(arr.astype(np.uint8)).convert("HSV")).astype(np.float32)
        H,S,V = hsv[:,:,0], hsv[:,:,1], hsv[:,:,2]
        feats["mean_H"],feats["mean_S"],feats["mean_V"] = H.mean(),S.mean(),V.mean()
        feats["std_H"],feats["std_S"],feats["std_V"]    = H.std(),S.std(),V.std()
        lab = np.array(Image.fromarray(arr.astype(np.uint8)).convert("LAB")).astype(np.float32)
        L,A,Bl = lab[:,:,0], lab[:,:,1], lab[:,:,2]
        feats["mean_L"],feats["mean_A"],feats["mean_Blab"] = L.mean(),A.mean(),Bl.mean()
        feats["std_L"],feats["std_A"],feats["std_Blab"]    = L.std(),A.std(),Bl.std()
        try:
            from scipy.ndimage import sobel, laplace
            sob = np.hypot(sobel(gray,0), sobel(gray,1))
            feats["edge_density"] = (sob>sob.mean()).mean()
            feats["lap_var"] = laplace(gray).var()
        except Exception:
            pass
        hist,_ = np.histogram(gray, bins=64, range=(0,255), density=True)
        hist = hist + 1e-12
        feats["entropy"] = float(-(hist*np.log(hist)).sum())
    except Exception:
        pass
    return feats

def safe_fill(df: pd.DataFrame) -> pd.DataFrame:
    df = df.replace([np.inf,-np.inf], np.nan)
    df = df.fillna(df.mean()).fillna(0)
    return df


In [ ]:
def predict_tabular(train_df: pd.DataFrame,
                    y_wide_df: pd.DataFrame,
                    test_df: pd.DataFrame,
                    target_names):
    train_images = y_wide_df.index.tolist()
    feats_train = []
    for rel in train_images:
        f = extract_image_features(rel); f["image_path"] = rel; feats_train.append(f)
    X_train_df = pd.DataFrame(feats_train).set_index("image_path")
    X_train_df = safe_fill(X_train_df)

    X = X_train_df.values
    Y = y_wide_df[target_names].values
    Y_log = np.log1p(Y)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    oof_pred_log = np.zeros_like(Y_log)
    for fold, (tr, va) in enumerate(kf.split(X)):
        X_tr, X_va = X[tr], X[va]
        Y_tr, Y_va = Y_log[tr], Y_log[va]

        preds_log_va = np.zeros_like(Y_va)
        for j, t in enumerate(target_names):
            w = np.full(len(X_tr), TARGET_WEIGHTS[t], dtype=np.float32)
            hgb = HistGradientBoostingRegressor(
                max_depth=4, learning_rate=0.1, max_iter=400, random_state=42
            )
            hgb.fit(X_tr, Y_tr[:, j], sample_weight=w)
            preds_log_va[:, j] = hgb.predict(X_va)

        oof_pred_log[va] = preds_log_va

        df_true = (pd.DataFrame(np.expm1(Y_va), columns=target_names)
                   .reset_index().melt(id_vars="index", var_name="target_name", value_name="y_true"))
        df_pred = (pd.DataFrame(np.expm1(preds_log_va), columns=target_names)
                   .reset_index().melt(id_vars="index", var_name="target_name", value_name="y_pred"))
        df = df_true.merge(df_pred, on=["index","target_name"]).drop(columns="index")
        print(f"[TAB] Fold {fold} R2w:", f"{weighted_r2_from_long(df):.4f}")

    df_true = (pd.DataFrame(np.expm1(Y_log), columns=target_names)
               .reset_index().melt(id_vars="index", var_name="target_name", value_name="y_true"))
    df_pred = (pd.DataFrame(np.expm1(oof_pred_log), columns=target_names)
               .reset_index().melt(id_vars="index", var_name="target_name", value_name="y_pred"))
    df_cv = df_true.merge(df_pred, on=["index","target_name"]).drop(columns="index")
    print("[TAB] CV R2w global:", f"{weighted_r2_from_long(df_cv):.4f}")

    calib = {}
    for j, t in enumerate(target_names):
        ridge = Ridge(alpha=1e-6, fit_intercept=True)
        ridge.fit(np.expm1(oof_pred_log[:, [j]]), np.expm1(Y_log[:, j:j+1]))
        calib[t] = (float(ridge.coef_[0][0]), float(ridge.intercept_[0]))

    models = []
    for j, t in enumerate(target_names):
        w = np.full(len(X), TARGET_WEIGHTS[t], dtype=np.float32)
        hgb = HistGradientBoostingRegressor(
            max_depth=4, learning_rate=0.1, max_iter=400, random_state=42
        )
        hgb.fit(X, Y_log[:, j], sample_weight=w)
        models.append(hgb)

    test_images = test_df["image_path"].unique().tolist()
    feats_test = []
    for rel in test_images:
        f = extract_image_features(rel); f["image_path"] = rel; feats_test.append(f)
    X_test_df = pd.DataFrame(feats_test).set_index("image_path")
    X_test_df = safe_fill(X_test_df)
    X_test = X_test_df.values

    preds_log = np.column_stack([m.predict(X_test) for m in models])
    preds = np.expm1(preds_log); preds = np.clip(preds, 0, None)

    for j, t in enumerate(target_names):
        a, b = calib[t]
        preds[:, j] = a * preds[:, j] + b

    preds_tab = pd.DataFrame(preds, index=test_images, columns=target_names)
    return preds_tab, oof_pred_log


In [ ]:

class CSIROTrainDataset(Dataset):
    def __init__(self, image_paths, targets_log, transform=None):
        self.image_paths = image_paths; self.targets_log = targets_log; self.transform = transform
    def __len__(self): return len(self.image_paths)
    def _safe(self, p):
        pth = COMP_DIR / p
        try:
            with Image.open(pth) as im: im = im.convert("RGB")
        except Exception: im = Image.new("RGB",(224,224),(0,0,0))
        return im
    def __getitem__(self, i):
        im = self._safe(self.image_paths[i])
        if self.transform: im = self.transform(im)
        y = torch.tensor(self.targets_log[i], dtype=torch.float32)
        return im, y

class CSIROTestDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths; self.transform = transform
    def __len__(self): return len(self.image_paths)
    def _safe(self, p):
        pth = COMP_DIR / p
        try:
            with Image.open(pth) as im: im = im.convert("RGB")
        except Exception: im = Image.new("RGB",(224,224),(0,0,0))
        return im
    def __getitem__(self, i):
        im = self._safe(self.image_paths[i])
        if self.transform: im = self.transform(im)
        return im

class SmallCNN(nn.Module):
    def __init__(self, n_outputs):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1), nn.ReLU(), nn.AdaptiveAvgPool2d((1,1))
        )
        self.head = nn.Sequential(nn.Flatten(), nn.Linear(128,128), nn.ReLU(), nn.Linear(128,n_outputs))
    def forward(self,x): return self.head(self.features(x))

def get_transforms():
    train_tf = T.Compose([T.Resize((224,224)), T.RandomHorizontalFlip(0.5), T.ToTensor()])
    test_tf  = T.Compose([T.Resize((224,224)), T.ToTensor()])
    return train_tf, test_tf

def tta_predict(model, image_tensor, n=4):
    x = image_tensor.unsqueeze(0).to(device)
    outs = []
    with torch.no_grad():
        outs.append(model(x).cpu().numpy()[0])
        outs.append(model(torch.flip(x,dims=[3])).cpu().numpy()[0])
        outs.append(model(torch.flip(x,dims=[2])).cpu().numpy()[0])
        outs.append(model(torch.rot90(x,k=1,dims=[2,3])).cpu().numpy()[0])
    return np.mean(outs[:n], axis=0)

def cnn_oof_onefold(y_wide_df, target_names, epochs=4, batch_size=16, lr=1e-3, wd=1e-5):
    paths = y_wide_df.index.to_numpy()
    n = len(paths); cut = int(0.8*n)
    tr_idx, va_idx = np.arange(cut), np.arange(cut, n)
    Y = y_wide_df.values; Y_log = np.log1p(Y)

    train_tf, _ = get_transforms()
    ds_tr = CSIROTrainDataset(paths[tr_idx], Y_log[tr_idx], transform=train_tf)
    ds_va = CSIROTrainDataset(paths[va_idx], Y_log[va_idx], transform=train_tf)
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    model = SmallCNN(n_outputs=len(target_names)).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    w = torch.tensor(W_VEC, device=device).view(1,-1)
    idx_total  = target_names.index("Dry_Total_g")
    idx_green  = target_names.index("Dry_Green_g")
    idx_dead   = target_names.index("Dry_Dead_g")
    idx_clover = target_names.index("Dry_Clover_g")
    lambda_cons = 0.3

    for ep in range(epochs):
        model.train()
        for xb, yb in dl_tr:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            pred = model(xb)  # log1p
            base = ((pred - yb)**2 * w).mean()
            # Consistency in ORIGINAL scale
            pred_lin  = torch.expm1(pred)
            sum_parts = pred_lin[:, idx_green] + pred_lin[:, idx_dead] + pred_lin[:, idx_clover]
            cons      = ((pred_lin[:, idx_total] - sum_parts)**2).mean()
            (base + lambda_cons*cons).backward(); opt.step()
        print(f"[CNN OOF] epoch {ep+1}/{epochs}")

    model.eval(); preds=[]
    with torch.no_grad():
        for xb, yb in dl_va:
            xb = xb.to(device); preds.append(model(xb).cpu().numpy())
    preds = np.concatenate(preds, axis=0)
    oof = np.zeros_like(Y_log); oof[va_idx] = preds
    return oof

def train_cnn_predict(y_wide_df, test_df, target_names,
                      epochs=8, batch_size=16, lr=1e-3, wd=1e-5, use_tta=True):
    img_paths = y_wide_df.index.tolist()
    Y = y_wide_df.values; Y_log = np.log1p(Y)

    train_tf, test_tf = get_transforms()
    ds = CSIROTrainDataset(img_paths, Y_log, transform=train_tf)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    model = SmallCNN(n_outputs=len(target_names)).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

    w = torch.tensor(W_VEC, device=device).view(1,-1)
    idx_total  = target_names.index("Dry_Total_g")
    idx_green  = target_names.index("Dry_Green_g")
    idx_dead   = target_names.index("Dry_Dead_g")
    idx_clover = target_names.index("Dry_Clover_g")
    lambda_cons = 0.3

    model.train()
    for ep in range(epochs):
        running=0.0
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            pred = model(xb)  # log1p
            base = ((pred - yb)**2 * w).mean()
            pred_lin  = torch.expm1(pred)
            sum_parts = pred_lin[:, idx_green] + pred_lin[:, idx_dead] + pred_lin[:, idx_clover]
            cons      = ((pred_lin[:, idx_total] - sum_parts)**2).mean()
            loss = base + lambda_cons*cons
            loss.backward(); opt.step()
            running += loss.item()*xb.size(0)
        print(f"[CNN] Epoch {ep+1}/{epochs} - loss_w(log1p + cons_orig): {running/len(ds):.4f}")

    test_images = test_df["image_path"].unique().tolist()
    ds_test = CSIROTestDataset(test_images, transform=test_tf)
    dl_test = DataLoader(ds_test, batch_size=1, shuffle=False, num_workers=2, pin_memory=True)

    model.eval(); preds_log=[]
    with torch.no_grad():
        for xb in dl_test:
            x = xb[0]
            if use_tta:
                pb = tta_predict(model, x, n=4)
                preds_log.append(pb[None, :])
            else:
                preds_log.append(model(x.unsqueeze(0).to(device)).cpu().numpy())
    preds_log = np.concatenate(preds_log, axis=0)
    preds = np.expm1(preds_log); preds = np.clip(preds, 0, None)
    preds_cnn = pd.DataFrame(preds, index=test_images, columns=target_names)
    return preds_cnn


In [ ]:
def learn_stacking_coeffs(oof_tab_log, oof_cnn_log, y_wide_df, target_names):
    mask_valid = (np.abs(oof_cnn_log).sum(axis=1) > 0)
    if mask_valid.sum() < 50:
        print("[STACK] Muy pocas filas válidas; uso blending fijo.")
        return None
    coefs = {}
    for j, t in enumerate(target_names):
        x_tab  = np.expm1(oof_tab_log[mask_valid, j])
        x_cnn  = np.expm1(oof_cnn_log[mask_valid, j])
        y_true = y_wide_df.values[mask_valid, j]
        Xs = np.column_stack([x_tab, x_cnn])
        ridge = Ridge(alpha=1e-6, fit_intercept=True).fit(Xs, y_true)
        coefs[t] = (float(ridge.coef_[0]), float(ridge.coef_[1]), float(ridge.intercept_))
    return coefs

def apply_stacking(preds_tab_df, preds_cnn_df, coefs, target_names):
    preds = preds_tab_df.copy()
    for t in target_names:
        a,b,c = coefs[t]
        preds[t] = a*preds_tab_df[t] + b*preds_cnn_df[t] + c
    return preds

def blend_total_consistency(preds_df, y_wide_df, target_names, gamma_soft=0.7):
    total_from_parts = preds_df["Dry_Green_g"] + preds_df["Dry_Dead_g"] + preds_df["Dry_Clover_g"]
    preds_df["Dry_Total_g"] = gamma_soft*preds_df["Dry_Total_g"] + (1-gamma_soft)*total_from_parts
    return preds_df

def make_submission_from_predictions(test_df: pd.DataFrame,
                                     preds_df: pd.DataFrame,
                                     target_names) -> pd.DataFrame:
    preds_long = (preds_df.reset_index()
                  .melt(id_vars="index", var_name="target_name", value_name="pred")
                  .rename(columns={"index":"image_path"}))
    test_pred = test_df.merge(preds_long, on=["image_path","target_name"], how="left")
    sub = test_pred[["sample_id","pred"]].rename(columns={"pred":"target"})
    assert sub.shape[0] == test_df.shape[0], "Submission size must match test size"
    assert np.isfinite(sub["target"]).all(), "NaN/Inf in submission"
    sub["target"] = sub["target"].clip(lower=0)
    return sub


In [ ]:
try:
    print(">> Training TABULAR ...")
    preds_tab, oof_tab_log = predict_tabular(train, y_wide, test, target_names)

    print("\n>> CNN OOF onefold (for stacking) ...")
    oof_cnn_log = cnn_oof_onefold(y_wide, target_names, epochs=4, batch_size=16, lr=1e-3, wd=1e-5)

    print("\n>> Training CNN (final) ...")
    preds_cnn = train_cnn_predict(y_wide, test, target_names,
                                  epochs=8, batch_size=16, lr=1e-3, wd=1e-5, use_tta=True)

    print("\n>> Learning stacking coefficients ...")
    coefs = learn_stacking_coeffs(oof_tab_log, oof_cnn_log, y_wide, target_names)

    if coefs is None:
        print(">> Applying fixed blending fallback ...")
        ALPHA = {"Dry_Green_g":0.50, "Dry_Dead_g":0.50, "Dry_Clover_g":0.50,
                 "GDM_g":0.60, "Dry_Total_g":0.70}
        preds_ens = preds_tab.copy()
        for t in target_names:
            a = ALPHA[t]
            preds_ens[t] = a*preds_tab[t] + (1-a)*preds_cnn[t]
    else:
        print(">> Applying learned stacking ...")
        preds_ens = apply_stacking(preds_tab, preds_cnn, coefs, target_names)

    print(">> Soft total consistency (0.7 direct + 0.3 parts) ...")
    preds_ens = blend_total_consistency(preds_ens, y_wide, target_names, gamma_soft=0.7)

    print(">> Building submission ...")
    submission = make_submission_from_predictions(test, preds_ens, target_names)
    print("Submission ready. Rows:", len(submission))
    display(submission.head())

except Exception as e:
    print("ERROR in pipeline; using fallback based on test. Error:", repr(e))
    submission = test[["sample_id"]].copy()
    submission["target"] = float(train["target"].mean())

assert submission.shape[0] == test.shape[0], "Submission size mismatch"
assert np.isfinite(submission["target"]).all(), "NaN/Inf in submission"
submission["target"] = submission["target"].clip(lower=0)

submission.to_csv("submission.csv", index=False)
print("submission.csv written.")
